# AI Agent Development with CrewAI

## What is CrewAI?

**CrewAI** is a role-based multi-agent orchestration framework that makes it easy to build teams of AI agents that collaborate to accomplish complex tasks.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Agent** | An autonomous unit with a specific role, goal, and backstory. Each agent can use tools and an LLM to reason and act. |
| **Task** | A unit of work with a description and expected output, assigned to an agent. |
| **Crew** | A team of agents working together on a collection of tasks. |
| **Process** | The execution strategy — Sequential (tasks run in order) or Hierarchical (a manager delegates). |
| **Tools** | Functions that agents can call to interact with the outside world (search, scrape, calculate, etc.). |

This tutorial walks through building progressively more complex agent systems with CrewAI, using OpenAI as the LLM provider.

**Prerequisites**: Familiarity with Python, LLM APIs, and basic prompt engineering.

---
## Section 1: Installation & Setup

In [ ]:
# Install CrewAI and its tools extension
!pip install -q crewai 'crewai[tools]'

In [ ]:
import os
import getpass

# Set the OpenAI API key from environment, with interactive fallback
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("OpenAI API key is set.")

---
## Section 2: Core Components Deep Dive

Before building anything, let's understand each building block in detail.

### 2.1 Agent

An Agent is defined by:

- **`role`**: A short title (e.g., "Senior Researcher"). This shapes how the LLM behaves.
- **`goal`**: What the agent is trying to achieve. Focuses the agent's reasoning on a specific objective.
- **`backstory`**: A narrative persona. This is surprisingly important — it gives the LLM context for *how* to approach problems, what expertise to draw on, and what tone to use.
- **`tools`**: A list of tool objects the agent can invoke.
- **`llm`**: The language model to use (defaults to OpenAI's GPT-4o).
- **`verbose`**: Whether to print the agent's reasoning steps.

In [ ]:
from crewai import Agent

# Example: defining an agent (not executed in a crew yet)
example_agent = Agent(
    role="Senior Data Scientist",
    goal="Analyze datasets and extract actionable insights",
    backstory=(
        "You are a veteran data scientist with 15 years of experience "
        "at top tech companies. You specialize in finding patterns that "
        "others miss and communicating complex findings clearly."
    ),
    tools=[],       # no tools for this example
    verbose=True,
)

print(f"Agent role: {example_agent.role}")
print(f"Agent goal: {example_agent.goal}")

### 2.2 Task

A Task is a specific unit of work:

- **`description`**: What needs to be done — the more specific, the better the output.
- **`expected_output`**: What the final deliverable should look like. This acts as a quality rubric.
- **`agent`**: Which agent is responsible (optional in hierarchical mode).

In [ ]:
from crewai import Task

# Example task definition
example_task = Task(
    description=(
        "Analyze the provided customer churn dataset. "
        "Identify the top 3 factors contributing to churn "
        "and suggest data-driven retention strategies."
    ),
    expected_output=(
        "A structured report with: (1) top 3 churn factors with evidence, "
        "(2) three actionable retention strategies, (3) recommended KPIs to track."
    ),
    agent=example_agent,
)

print(f"Task description: {example_task.description[:80]}...")

### 2.3 Crew

A **Crew** is the team — it brings agents and tasks together and runs them.

```python
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    process=Process.sequential,
    verbose=True,
)
result = crew.kickoff()
```

### 2.4 Process

| Process | How It Works | When to Use |
|---------|-------------|-------------|
| **Sequential** | Tasks execute in the order they are listed. Output from task N is available to task N+1. | Predictable pipelines: research → analyze → write |
| **Hierarchical** | A manager agent receives all tasks and dynamically delegates them to the best-suited agent. | When you want flexible delegation, or tasks have ambiguous ownership |

### 2.5 Tools

Tools let agents interact with the world. CrewAI supports:

1. **Built-in tools** from `crewai_tools` (e.g., `SerperDevTool`, `ScrapeWebsiteTool`)
2. **Custom tools** via the `@tool` decorator (quick and simple)
3. **Custom tools** via `BaseTool` class (more structure, validation, reusability)

In [ ]:
from crewai.tools import tool, BaseTool

# Quick custom tool using the @tool decorator
@tool("Quick Calculator")
def quick_calc(expression: str) -> str:
    """Evaluates a simple math expression and returns the result."""
    # Mock implementation for safety
    return f"The result of '{expression}' is 42 (mock result)"

print(f"Tool name: {quick_calc.name}")
print(f"Tool description: {quick_calc.description}")

---
## Section 3: Build a Simple Agent (Single Agent + Tool)

Let's build our first working crew: a single research agent with a mock search tool.

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool

# ── Mock Search Tool ─────────────────────────────────────────────
# This returns fake results so we don't need a real search API key.
# In production, you would use SerperDevTool or a real API.

@tool("Search Tool")
def search_tool(query: str) -> str:
    """Searches the web for information on the given query and returns relevant results."""
    mock_results = {
        "default": (
            "Search results for: '{query}'\n"
            "1. AI agents are software entities that perceive their environment "
            "and take actions to achieve goals. They use LLMs for reasoning.\n"
            "2. Multi-agent systems coordinate multiple agents, each specialized "
            "in different tasks, to solve complex problems collaboratively.\n"
            "3. Key frameworks: CrewAI (role-based), LangGraph (graph-based), "
            "AutoGen (conversation-based). Each has different strengths.\n"
            "4. Best practices include: clear role definitions, structured outputs, "
            "tool use for grounding, and iterative refinement of prompts."
        )
    }
    return mock_results["default"].format(query=query)

print("Search tool created.")

In [ ]:
# ── Research Agent ────────────────────────────────────────────────
researcher = Agent(
    role="Senior Researcher",
    goal="Find comprehensive and accurate information on the given topic",
    backstory=(
        "You are an expert researcher with a PhD in Computer Science. "
        "You are meticulous about fact-checking and always look for "
        "primary sources. You excel at synthesizing information from "
        "multiple sources into clear, well-organized summaries."
    ),
    tools=[search_tool],
    verbose=True,
)

# ── Research Task ─────────────────────────────────────────────────
research_task = Task(
    description=(
        "Research the current state of AI agent frameworks. "
        "Focus on: what AI agents are, how multi-agent systems work, "
        "and what the leading frameworks are. Use the search tool "
        "to gather information."
    ),
    expected_output=(
        "A detailed research summary with: "
        "(1) Definition of AI agents, "
        "(2) Overview of multi-agent systems, "
        "(3) Comparison of top 3 frameworks, "
        "(4) Key best practices. "
        "Format as a structured report with headers."
    ),
    agent=researcher,
)

# ── Crew ───────────────────────────────────────────────────────────
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,
    verbose=True,
)

print("Crew assembled. Ready to kick off.")

In [ ]:
# ── Run the crew ──────────────────────────────────────────────────
result = crew.kickoff()

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print(result.raw)

In [ ]:
# ── Inspect the result object ─────────────────────────────────────
print("Result type:", type(result))
print("\nRaw output (first 500 chars):")
print(result.raw[:500] if len(result.raw) > 500 else result.raw)

# Check available attributes on the result
print("\nAvailable attributes:")
print([attr for attr in dir(result) if not attr.startswith("_")])

---
## Section 4: Multi-Agent Sequential Crew

Now let's build a three-agent pipeline:

1. **Researcher** — gathers information using the search tool
2. **Analyst** — structures and analyzes the findings
3. **Writer** — produces a polished final report

In `Process.sequential`, each task's output feeds into the next task as context.

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool

# ── Mock Search Tool (reused) ─────────────────────────────────────
@tool("Search Tool")
def search_tool_v2(query: str) -> str:
    """Searches the web for information on the given query and returns relevant results."""
    return (
        f"Search results for: '{query}'\n"
        "1. The global AI market is projected to reach $1.8 trillion by 2030.\n"
        "2. Key growth drivers: generative AI adoption, enterprise automation, "
        "and advances in reasoning capabilities.\n"
        "3. Major players: OpenAI, Google DeepMind, Anthropic, Meta AI, Mistral.\n"
        "4. Emerging trends: AI agents, multimodal models, on-device AI, "
        "and AI-powered scientific discovery.\n"
        "5. Challenges: regulation, safety, energy consumption, job displacement.\n"
        "6. Enterprise adoption is accelerating — 72% of Fortune 500 companies "
        "now have dedicated AI teams."
    )

# ── Agent 1: Researcher ────────────────────────────────────────────
researcher = Agent(
    role="Senior Research Analyst",
    goal="Gather comprehensive data and facts on the given topic",
    backstory=(
        "You are an investigative researcher who leaves no stone unturned. "
        "You have a talent for finding the most relevant and up-to-date "
        "information and organizing it systematically."
    ),
    tools=[search_tool_v2],
    verbose=True,
)

# ── Agent 2: Analyst ──────────────────────────────────────────────
analyst = Agent(
    role="Strategic Analyst",
    goal="Analyze research findings and identify key insights, trends, and implications",
    backstory=(
        "You are a seasoned strategic analyst at a top consulting firm. "
        "You excel at seeing the big picture, identifying patterns, "
        "and drawing actionable conclusions from raw data."
    ),
    tools=[],  # The analyst works with the researcher's output, no external tools needed
    verbose=True,
)

# ── Agent 3: Writer ───────────────────────────────────────────────
writer = Agent(
    role="Content Writer",
    goal="Produce a polished, engaging, and well-structured report",
    backstory=(
        "You are an award-winning technical writer who makes complex "
        "topics accessible. You have a gift for clear prose, logical "
        "structure, and compelling narratives."
    ),
    tools=[],
    verbose=True,
)

print("All 3 agents created.")

In [ ]:
# ── Task 1: Research ──────────────────────────────────────────────
research_task = Task(
    description=(
        "Research the current state and future outlook of the AI industry. "
        "Use the search tool to find information about market size, key players, "
        "emerging trends, and challenges. Compile all findings."
    ),
    expected_output=(
        "A comprehensive collection of facts and data points covering: "
        "market size and projections, major companies, key trends, "
        "and industry challenges. Include specific numbers where available."
    ),
    agent=researcher,
)

# ── Task 2: Analysis ──────────────────────────────────────────────
analysis_task = Task(
    description=(
        "Analyze the research findings from the previous task. "
        "Identify the top 3 most impactful trends, assess risks, "
        "and provide strategic recommendations for a tech company "
        "looking to invest in AI."
    ),
    expected_output=(
        "A structured analysis containing: "
        "(1) Executive summary, "
        "(2) Top 3 trends with impact assessment, "
        "(3) Risk analysis, "
        "(4) Three strategic recommendations with rationale."
    ),
    agent=analyst,
)

# ── Task 3: Writing ───────────────────────────────────────────────
writing_task = Task(
    description=(
        "Using the analysis provided, write a polished executive briefing "
        "on the AI industry outlook. The briefing should be professional, "
        "concise, and suitable for C-suite executives."
    ),
    expected_output=(
        "A 400-600 word executive briefing with: title, executive summary, "
        "key findings section, strategic recommendations section, "
        "and a conclusion. Professional tone throughout."
    ),
    agent=writer,
)

print("All 3 tasks defined.")

In [ ]:
# ── Assemble and run the Sequential Crew ──────────────────────────
sequential_crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task],
    process=Process.sequential,  # Tasks run in order: research → analyze → write
    verbose=True,
)

sequential_result = sequential_crew.kickoff()

print("\n" + "=" * 60)
print("FINAL OUTPUT (from Writer)")
print("=" * 60)
print(sequential_result.raw)

In [ ]:
# ── Inspect per-task results ──────────────────────────────────────
# tasks_output gives you the output of each individual task
if hasattr(sequential_result, "tasks_output"):
    for i, task_output in enumerate(sequential_result.tasks_output):
        print(f"\n{'=' * 60}")
        print(f"Task {i + 1} Output (first 300 chars):")
        print(f"{'=' * 60}")
        raw = task_output.raw if hasattr(task_output, "raw") else str(task_output)
        print(raw[:300] + "..." if len(raw) > 300 else raw)
else:
    print("tasks_output attribute not available on this result object.")

---
## Section 5: Multi-Agent Hierarchical Crew

In **hierarchical mode**, a manager agent is automatically created. Instead of tasks running in a fixed order, the manager:

1. Receives all tasks
2. Evaluates which agent is best suited for each task
3. Delegates dynamically
4. May re-delegate or ask for revisions

Key differences from sequential:
- Tasks do **not** need an explicit `agent` assignment (the manager decides)
- You must provide a `manager_llm` for the manager agent to use
- Execution order is flexible and determined at runtime

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_openai import ChatOpenAI

# ── Redefine agents (same roles, no changes needed) ───────────────
@tool("Search Tool")
def search_tool_v3(query: str) -> str:
    """Searches the web for information on the given query and returns relevant results."""
    return (
        f"Search results for: '{query}'\n"
        "1. AI agents market expected to grow at 43% CAGR through 2028.\n"
        "2. Enterprise adoption focuses on customer service, code generation, "
        "and data analysis use cases.\n"
        "3. Open-source models closing the gap with proprietary ones.\n"
        "4. Regulatory frameworks emerging in EU, US, and China."
    )

h_researcher = Agent(
    role="Senior Research Analyst",
    goal="Gather comprehensive data and facts on the given topic",
    backstory=(
        "You are an investigative researcher who leaves no stone unturned. "
        "You specialize in technology market research."
    ),
    tools=[search_tool_v3],
    verbose=True,
)

h_analyst = Agent(
    role="Strategic Analyst",
    goal="Analyze findings and extract actionable insights",
    backstory=(
        "You are a strategic analyst with deep expertise in technology "
        "markets and business strategy."
    ),
    tools=[],
    verbose=True,
)

h_writer = Agent(
    role="Executive Report Writer",
    goal="Produce polished, executive-ready content",
    backstory=(
        "You are a professional writer who specializes in C-suite "
        "communications and executive briefings."
    ),
    tools=[],
    verbose=True,
)

print("Hierarchical agents created.")

In [ ]:
# ── Define tasks WITHOUT agent assignments ────────────────────────
# In hierarchical mode, the manager decides who does what.

h_research_task = Task(
    description=(
        "Research the AI agents ecosystem: market size, growth rate, "
        "key players, and adoption trends. Use search tools to gather data."
    ),
    expected_output="Comprehensive research findings with data points and sources.",
)

h_analysis_task = Task(
    description=(
        "Analyze the research findings. Identify the 3 most significant "
        "opportunities and 2 key risks in the AI agents space."
    ),
    expected_output="Structured analysis with opportunities, risks, and recommendations.",
)

h_writing_task = Task(
    description=(
        "Write a concise executive memo summarizing the AI agents opportunity. "
        "Keep it under 400 words with clear section headers."
    ),
    expected_output="A polished executive memo with headers, key points, and a recommendation.",
)

print("Hierarchical tasks defined (no agent assignments).")

In [ ]:
# ── Assemble Hierarchical Crew ────────────────────────────────────
hierarchical_crew = Crew(
    agents=[h_researcher, h_analyst, h_writer],
    tasks=[h_research_task, h_analysis_task, h_writing_task],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o"),  # The manager uses this LLM
    verbose=True,
)

hierarchical_result = hierarchical_crew.kickoff()

print("\n" + "=" * 60)
print("HIERARCHICAL CREW RESULT")
print("=" * 60)
print(hierarchical_result.raw)

### Sequential vs Hierarchical: Trade-offs

| Aspect | Sequential | Hierarchical |
|--------|-----------|---------------|
| **Predictability** | High — tasks always run in the same order | Lower — manager decides at runtime |
| **Flexibility** | Low — fixed pipeline | High — manager can re-delegate, iterate |
| **Cost** | Lower — no manager LLM calls | Higher — extra LLM calls for manager reasoning |
| **Debugging** | Easier — linear flow | Harder — dynamic delegation |
| **Best for** | Well-defined pipelines | Ambiguous task ownership, complex workflows |

**Rule of thumb**: Start with sequential. Switch to hierarchical when you need the manager's judgment about task routing.

---
## Section 6: Custom Tools

CrewAI offers two ways to build custom tools:

1. **`@tool` decorator** — quick, minimal boilerplate, great for simple functions
2. **`BaseTool` class** — more structured, supports validation, better for complex or reusable tools

In [ ]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

# ── Custom Tool via BaseTool class ────────────────────────────────

class WebScraperInput(BaseModel):
    """Input schema for the WebScraperTool."""
    url: str = Field(description="The URL of the website to scrape")


class WebScraperTool(BaseTool):
    name: str = "Web Scraper"
    description: str = (
        "Scrapes content from a given URL and returns the main text content. "
        "Use this when you need to extract information from a specific web page."
    )
    args_schema: Type[BaseModel] = WebScraperInput

    def _run(self, url: str) -> str:
        """Mock implementation that returns fake scraped data."""
        return (
            f"Scraped content from {url}:\n\n"
            "Title: The Future of AI-Powered Automation\n"
            "Content: AI-powered automation is transforming industries at an "
            "unprecedented pace. Companies leveraging AI agents report 40% "
            "improvement in operational efficiency. Key sectors include: "
            "healthcare (diagnostic AI), finance (fraud detection), "
            "manufacturing (predictive maintenance), and customer service "
            "(AI chatbots handling 65% of routine inquiries).\n"
            "Published: 2025 | Source: Tech Industry Report"
        )


# Instantiate the tool
web_scraper = WebScraperTool()

print(f"BaseTool name: {web_scraper.name}")
print(f"BaseTool description: {web_scraper.description[:80]}...")

In [ ]:
from crewai.tools import tool

# ── Custom Tool via @tool decorator ───────────────────────────────

@tool("Sentiment Analyzer")
def sentiment_analyzer(text: str) -> str:
    """Analyzes the sentiment of the given text and returns a sentiment score and label."""
    # Mock sentiment analysis
    word_count = len(text.split())
    return (
        f"Sentiment Analysis Result:\n"
        f"- Text length: {word_count} words\n"
        f"- Overall sentiment: Positive (0.78/1.0)\n"
        f"- Confidence: 92%\n"
        f"- Key signals: optimistic language, forward-looking statements\n"
        f"- Emotional tone: Professional, confident"
    )

print(f"@tool name: {sentiment_analyzer.name}")

In [ ]:
from crewai import Agent, Task, Crew, Process

# ── Agent using both custom tools ─────────────────────────────────
research_analyst = Agent(
    role="Research & Sentiment Analyst",
    goal=(
        "Scrape web content and analyze its sentiment to provide "
        "comprehensive intelligence reports"
    ),
    backstory=(
        "You are a digital intelligence analyst who combines web research "
        "with sentiment analysis to provide nuanced reports. You always "
        "scrape relevant pages first, then analyze the sentiment of the content."
    ),
    tools=[web_scraper, sentiment_analyzer],
    verbose=True,
)

# ── Task that exercises both tools ────────────────────────────────
intel_task = Task(
    description=(
        "Investigate the topic of AI automation in enterprise. "
        "First, scrape the page at 'https://example.com/ai-automation' "
        "to gather content. Then, run sentiment analysis on the scraped "
        "content. Finally, produce a brief intelligence summary combining "
        "the factual findings with the sentiment assessment."
    ),
    expected_output=(
        "An intelligence brief with: (1) key facts from the scraped content, "
        "(2) sentiment assessment, (3) overall assessment of the topic's outlook."
    ),
    agent=research_analyst,
)

# ── Run it ─────────────────────────────────────────────────────────
tool_demo_crew = Crew(
    agents=[research_analyst],
    tasks=[intel_task],
    process=Process.sequential,
    verbose=True,
)

tool_result = tool_demo_crew.kickoff()

print("\n" + "=" * 60)
print("INTELLIGENCE BRIEF")
print("=" * 60)
print(tool_result.raw)

---
## Section 7: Key Takeaways & Framework Comparison

### When to Use CrewAI

CrewAI is an excellent choice when:

- You want to model your system as a **team of specialists** with distinct roles
- You need **rapid prototyping** — going from idea to working multi-agent system in minutes
- Your workflow maps naturally to **delegation patterns** (researcher → analyst → writer)
- You want **less boilerplate** than lower-level frameworks
- You need built-in support for **sequential and hierarchical** execution

### CrewAI vs LangGraph

| Dimension | CrewAI | LangGraph |
|-----------|--------|-----------|
| **Abstraction level** | High — role-based agents | Low — state machines and graphs |
| **Mental model** | Team of people with roles | Graph of nodes and edges |
| **Speed to prototype** | Very fast | Slower (more setup) |
| **Fine-grained control** | Limited | Full control over state and transitions |
| **Execution patterns** | Sequential, Hierarchical | Any graph topology (cycles, branches, etc.) |
| **Tool integration** | Built-in tool ecosystem | Manual tool binding |
| **Debugging** | Agent-level logs | State-level inspection |
| **Best for** | Role-based teams, content pipelines, rapid prototyping | Complex state machines, conditional logic, production systems |

### Summary

- **Start with CrewAI** if you want to get a multi-agent system running quickly and your workflow is role-based.
- **Move to LangGraph** when you need precise control over execution flow, complex branching, or custom state management.
- Both frameworks can use the same underlying LLMs and many of the same tools — the choice is about **orchestration philosophy**, not capability.